In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

class VerticalImageClassifier(nn.Module):
    def __init__(self, num_classes=361):
        super(VerticalImageClassifier, self).__init__()
        self.base_model = models.resnet18(weights='IMAGENET1K_V1')
        self.base_model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)  # adapt for 3-channel input
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)


In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((400, 200)),  # Match the synthetic image size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

dataset = datasets.ImageFolder(root='training_data', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)



In [3]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VerticalImageClassifier(num_classes=361).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):  # Adjust as needed
    model.train()
    running_loss = 0.0
    correct = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    accuracy = 100 * correct / len(dataset)
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\isatyamks/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:02<00:00, 20.7MB/s]


Epoch 1, Loss: 366.3761, Accuracy: 2.95%
Epoch 2, Loss: 304.0107, Accuracy: 13.50%
Epoch 3, Loss: 259.1966, Accuracy: 26.30%
Epoch 4, Loss: 219.7125, Accuracy: 43.70%
Epoch 5, Loss: 180.3483, Accuracy: 60.20%
Epoch 6, Loss: 143.3228, Accuracy: 74.65%
Epoch 7, Loss: 109.7042, Accuracy: 84.70%
Epoch 8, Loss: 80.7930, Accuracy: 91.50%
Epoch 9, Loss: 58.9144, Accuracy: 95.60%
Epoch 10, Loss: 41.3902, Accuracy: 98.25%


In [4]:
torch.save(model.state_dict(), "metal_plate_classifier.pth")


In [17]:
image_path  = 'testing_data\\2.jpg'

In [6]:
image_path

'testing_data\\1.jpg'

In [7]:
def predict(image_path, model, class_names):
    from PIL import Image
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return class_names[predicted.item()]


In [13]:
def predict(image_path, model, class_names, device='cuda' if torch.cuda.is_available() else 'cpu'):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match model input
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)               # Move image to same device as model
    model = model.to(device)               # Ensure model is on correct device
    model.eval()

    with torch.no_grad():
        outputs = model(image)
        _, predicted_idx = torch.max(outputs, 1)

    predicted_label = class_names[predicted_idx.item()]
    return predicted_label


In [14]:
class_names = [f"A{str(i).zfill(4)}" for i in range(361)]


In [20]:
image_path = 'testing_data//1.jpg'  # Path to your uploaded test image
predicted_label = predict(image_path, model, class_names)

print(predicted_label)

A0314
